# Block Ciphers

In [58]:
class F2m:
    def __init__(self, value, G):
        self.value = value
        self.G = G

    def __add__(self, other):
        return F2m(self.value ^ other.value, self.G)

    def __mul__(self, other):
        # Multiplication modulo G
        result = 0
        a = self.value
        b = other.value
        while b:
            if b & 1:
                result ^= a
            a <<= 1
            if a & (1 << len(bin(self.G)) - 3):
                a ^= self.G
            b >>= 1
        return F2m(result, self.G)

    def __repr__(self):
        return bin(self.value)[2:]


In [110]:
def f2m_add(a, b):
    # Addition modulo G
    return a ^ b

def f2m_mul(a, b, G):
    # Multiplication modulo G
    result = 0
    while b:
        if b & 1:
            result ^= a
        a <<= 1
        if a & (1 << len(bin(G)) - 3):
            a ^= G
        b >>= 1
    return result

In [109]:
G = 0b100011011  # x^8 + x^4 + x^3 + x + 1

b0 = F2m(0xEF, G)
b5 = F2m(0x19, G)
b10 = F2m(0x3A, G)
b15 = F2m(0x45, G)

_01 = F2m(0x1, G)
_02 = F2m(0x2, G)
_03 = F2m(0x3, G)

(_02 * b0), (_03 * b5), (_01 * b10), (_01 * b15)

(11000101, 101011, 111010, 1000101)

## AES

La entrada, un bloque A de 128 bits organizada como una matriz de 4x4 donde cada entrada contine un byte

In [103]:
A = [
    [0x00, 0x01, 0x02, 0x03],
    [0x04, 0x05, 0x06, 0x07],
    [0x08, 0x09, 0x0A, 0x0B],
    [0x0C, 0x0D, 0x0E, 0x0F],
]

### Byte Substitution

16 funciones identicas, biyenctivas y que representa la parte no lineal de AES. Son implementadas mediantes la tabla de busqueda AES S-box

In [22]:
s_box = [
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
]
s_box_inv = [
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D
] 

In [157]:
s_box_dict = dict(zip(s_box, s_box_inv)) 

def byte_substitution(A): 
    B = []

    for row in A:
        B.append([s_box_dict[x] for x in row])  
    return B

In [131]:
B = byte_substitution(A)
B

[[72, 64, 88, 181], [8, 36, 41, 118], [244, 114, 113, 223], [145, 126, 13, 99]]

### Shift rows

In [132]:
def shift_rows(B): 
    return [
        B[0],
        [B[1][1], B[1][2], B[1][3], B[1][0]],
        [B[2][2], B[2][3], B[2][0], B[2][1]],
        [B[3][3], B[3][0], B[2][1], B[2][2]],
    ]

B = shift_rows(B)
B

[[72, 64, 88, 181],
 [36, 41, 118, 8],
 [113, 223, 244, 114],
 [99, 145, 114, 113]]

### Mix Collumn
Realiza una transformación linal que mezcla cada columna de la matriz estado. Toda la artimetica es realizada en el campo $F_2^8$ utilizando el polinomi irreducible $x^8 + x^4 + x^3 + x + 1$

In [121]:
const_matrix = [
    [0x02, 0x03, 0x01, 0x01],
    [0x01, 0x02, 0x03, 0x01],
    [0x01, 0x01, 0x02, 0x03],
    [0x03, 0x01, 0x01, 0x02],
]

G = 0b100011011 

def mix_collumns(B): 
    C = [[0 for x in range(4)] for y in range(4)] 

    for i in range(4):
        for j in range(4):
            for k in range(4):
                  C[i][j] = f2m_add(C[i][j], f2m_mul(const_matrix[i][k], B[k][j], G))
    return C

C = mix_collumns(B)
C

[[237, 224, 183, 87],
 [206, 224, 148, 64],
 [27, 2, 132, 195],
 [29, 106, 170, 171]]

### Key addition

Las entradas son la matriz de estado C de 4 x 4 (cadena de 128 bits). Y una subllave de la ronda ki de 128 bits genreda por la función de expansión de llaves. La salida es $C \oplus k_i$

In [21]:
def key_addition(C, ki): 
    return C ^ ki 

### Función de Expansión 

La función expansión de llaves toma la llave original (de longitud 128, 192 o 256 bits) y retorna subllaves (llaves de ronda) de 128 bits. 

In [24]:
RC = [
    0x00000001,
    0x00000010,
    0x00000100,
    0x00001000,
    0x00010000,
    0x00100000,
    0x01000000,
    0x10000000,
    0x00011011,
    0x00110110
]

In [209]:
def g(w, i): 
    # Chat got create the two lines bellow
    w = w.to_bytes(4, byteorder='big')
    v = [int.from_bytes(w[i:i+1], byteorder='big') for i in range(0, len(w))]   

    v = [v[1], v[2], v[3], v[0]]
    
    # byte subtituion 
    v = [s_box_dict[vi] for vi in v]

    # XOR
    v[0] ^= RC[i]
    
    return v

In [199]:
def expansion_key_128(k): 
    # TODO: first 4 words from k
    w = [0]*(44)
    
    for i in range(10):    
        w[4*i] = w[4 + (i - 1)] ^ g(w[4 * i - 1], i)
        w[4*i + 1] = w[4 * (i - 1) + 1] ^ w[4 * i]
        w[4*i + 2] = w[4 * (i - 1) + 2] ^ w[4 * i + 1]
        w[4*i + 3] = w[4 * (i - 1) + 3] ^ w[4 * i + 2]
    
        print(w[4*i], w[4*i + 1], w[4*i + 2], w[4*i + 3])

In [210]:
i = 5
w = 0x1A38B5EE
[hex(v) for v in g(w, i)]

['0x10000f', '0x7f', '0xf9', '0x64']